# Core EMI calculation function

In [12]:
# ---------------------------------------------
# EMI Calculator - Core Computation Function
# ---------------------------------------------
# This function computes:
#   - Monthly EMI
#   - Total payment over the tenure
#   - Total interest paid
#
# Inputs:
#   principal (float): Loan amount
#   tenure_months (int): Loan tenure in months
#   annual_interest_rate (float): Annual interest rate in percent (e.g., 9.5 for 9.5%)
#
# Formula (when monthly rate r > 0):
#   EMI = [P * r * (1 + r)^n] / [(1 + r)^n - 1]
# Where:
#   P = principal
#   r = annual_interest_rate / (12 * 100)
#   n = tenure_months
#
# If interest rate is 0, EMI = P / n
# ---------------------------------------------

def calculate_emi(principal: float, tenure_months: int, annual_interest_rate: float):
    # Basic validation for required positive values
    if principal <= 0 or tenure_months <= 0:
        raise ValueError("principal and tenure_months must be greater than 0")

    # Convert annual percentage rate to a monthly decimal rate
    monthly_rate = (annual_interest_rate / 100.0) / 12.0

    # Handle zero-interest loans separately to avoid division by zero
    if monthly_rate == 0:
        emi = principal / tenure_months
    else:
        factor = (1 + monthly_rate) ** tenure_months
        emi = principal * monthly_rate * factor / (factor - 1)

    total_payment = emi * tenure_months
    total_interest = total_payment - principal

    # Round for display; you can return unrounded values if you need exact math later
    return round(emi, 2), round(total_payment, 2), round(total_interest, 2)


# (Optional) Amortization schedule generator

In [16]:
# -------------------------------------------------------
# Amortization Schedule (Optional but useful)
# -------------------------------------------------------
# Produces a month-by-month breakdown of:
#   - EMI
#   - Interest component
#   - Principal component
#   - Remaining balance
#
# Returns a list of dicts (easy to print or convert to DataFrame).
# -------------------------------------------------------

def amortization_schedule(principal: float, tenure_months: int, annual_interest_rate: float):
    # Monthly rate as a decimal
    monthly_rate = (annual_interest_rate / 100.0) / 12.0

    # Get the computed EMI from the core function
    emi, _, _ = calculate_emi(principal, tenure_months, annual_interest_rate)

    balance = principal
    schedule = []

    for month in range(1, tenure_months + 1):
        # Interest portion for the current month (zero if rate is 0)
        interest_component = round(balance * monthly_rate, 2) if monthly_rate > 0 else 0.0

        # Principal portion is the rest of the EMI after interest
        principal_component = round(emi - interest_component, 2)

        # Adjust the final month to eliminate rounding residue
        if month == tenure_months:
            principal_component = round(balance, 2)
            emi_effective = round(principal_component + interest_component, 2)
        else:
            emi_effective = emi

        # Update remaining balance
        balance = round(balance - principal_component, 2)
        balance = max(balance, 0.0)  # Guard against negative due to rounding

        # Append a row for this month
        schedule.append({
            "Month": month,
            "EMI": emi_effective,
            "Interest": interest_component,
            "Principal": principal_component,
            "Balance": balance
        })

    return schedule


# Simple currency formatting helper (optional, for nicer output)

In [15]:
# ---------------------------------------------
# Currency formatting helper (Optional)
# ---------------------------------------------
# Formats numbers with commas and 2 decimals.
# You can change the symbol if you like.
# ---------------------------------------------

def fmt_money(x, symbol="₹"):
    return f"{symbol}{x:,.2f}"


# Text interface to run inside Jupyter (input → calculation → print results)

In [14]:
# -------------------------------------------------------
# Text Interface - Runs cleanly inside Jupyter Notebook
# -------------------------------------------------------
# Prompts the user for:
#   - Loan amount
#   - Tenure (months)
#   - Annual interest rate
# Then prints:
#   - Monthly EMI
#   - Total Payment
#   - Total Interest
# Optionally offers to print a short amortization table.
# -------------------------------------------------------

def run_emi_calculator():
    print("📟 Welcome to the EMI Calculator\n")

    # Collect user input from the console
    try:
        principal = float(input("Enter loan amount (e.g., 500000): ").strip())
        tenure_months = int(input("Enter loan tenure in months (e.g., 60): ").strip())
        annual_rate = float(input("Enter annual interest rate in % (e.g., 9.5): ").strip())

        if principal <= 0 or tenure_months <= 0:
            print("\n❌ Loan amount and tenure must be greater than 0.")
            return

    except ValueError:
        print("\n⚠️ Invalid input. Please enter numeric values only.")
        return

    # Compute EMI, total payment, and total interest
    emi, total_payment, total_interest = calculate_emi(principal, tenure_months, annual_rate)

    # Use currency formatter if available; else plain numbers
    try:
        princ_str = fmt_money(principal)
        emi_str = fmt_money(emi)
        tot_pay_str = fmt_money(total_payment)
        tot_int_str = fmt_money(total_interest)
    except NameError:
        princ_str = f"{principal:,.2f}"
        emi_str = f"{emi:,.2f}"
        tot_pay_str = f"{total_payment:,.2f}"
        tot_int_str = f"{total_interest:,.2f}"

    # Display summary
    print("\n📊 EMI Calculation Results")
    print("-" * 40)
    print(f"Loan Amount (Principal): {princ_str}")
    print(f"Tenure (Months):        {tenure_months}")
    print(f"Interest Rate (Annual): {annual_rate}%")
    print("-" * 40)
    print(f"Monthly EMI:            {emi_str}")
    print(f"Total Payment:          {tot_pay_str}")
    print(f"Total Interest:         {tot_int_str}")
    print("-" * 40)

    # Optional: Ask to show a short amortization table
    choice = input("Show first 12 rows of amortization schedule? (y/n): ").strip().lower()
    if choice == "y":
        schedule = amortization_schedule(principal, tenure_months, annual_rate)
        print("\n📅 Amortization Schedule (first 12 months)")
        print("-" * 60)
        print(f"{'Month':>5} | {'EMI':>12} | {'Interest':>10} | {'Principal':>10} | {'Balance':>12}")
        print("-" * 60)
        for row in schedule[:12]:
            # Format values nicely; fall back if fmt_money not defined
            try:
                emi_v = fmt_money(row['EMI'])
                int_v = fmt_money(row['Interest'])
                prin_v = fmt_money(row['Principal'])
                bal_v = fmt_money(row['Balance'])
            except NameError:
                emi_v = f"{row['EMI']:,.2f}"
                int_v = f"{row['Interest']:,.2f}"
                prin_v = f"{row['Principal']:,.2f}"
                bal_v = f"{row['Balance']:,.2f}"

            print(f"{row['Month']:>5} | {emi_v:>12} | {int_v:>10} | {prin_v:>10} | {bal_v:>12}")

        if len(schedule) > 12:
            print("... (schedule continues)")


# Run the text interface

In [17]:
# -----------------------------
# Execute the text interface
# -----------------------------
run_emi_calculator()


📟 Welcome to the EMI Calculator



Enter loan amount (e.g., 500000):  100000
Enter loan tenure in months (e.g., 60):  10
Enter annual interest rate in % (e.g., 9.5):  9.5



📊 EMI Calculation Results
----------------------------------------
Loan Amount (Principal): ₹100,000.00
Tenure (Months):        10
Interest Rate (Annual): 9.5%
----------------------------------------
Monthly EMI:            ₹10,440.57
Total Payment:          ₹104,405.66
Total Interest:         ₹4,405.66
----------------------------------------


Show first 12 rows of amortization schedule? (y/n):  y



📅 Amortization Schedule (first 12 months)
------------------------------------------------------------
Month |          EMI |   Interest |  Principal |      Balance
------------------------------------------------------------
    1 |   ₹10,440.57 |    ₹791.67 |  ₹9,648.90 |   ₹90,351.10
    2 |   ₹10,440.57 |    ₹715.28 |  ₹9,725.29 |   ₹80,625.81
    3 |   ₹10,440.57 |    ₹638.29 |  ₹9,802.28 |   ₹70,823.53
    4 |   ₹10,440.57 |    ₹560.69 |  ₹9,879.88 |   ₹60,943.65
    5 |   ₹10,440.57 |    ₹482.47 |  ₹9,958.10 |   ₹50,985.55
    6 |   ₹10,440.57 |    ₹403.64 | ₹10,036.93 |   ₹40,948.62
    7 |   ₹10,440.57 |    ₹324.18 | ₹10,116.39 |   ₹30,832.23
    8 |   ₹10,440.57 |    ₹244.09 | ₹10,196.48 |   ₹20,635.75
    9 |   ₹10,440.57 |    ₹163.37 | ₹10,277.20 |   ₹10,358.55
   10 |   ₹10,440.56 |     ₹82.01 | ₹10,358.55 |        ₹0.00
